In [1]:
from pathlib import Path
import os
import json
import copy
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

# 多分もう使わない

In [2]:
from feature import Feature

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
def json2data(path):
    cols = ["text", "label", "subLabel"]
    df = pd.DataFrame(index=[], columns=cols)
    files = os.listdir(path)
    for cop in files:
        if "." not in cop:
            continue
        with open(path+cop, "r") as f:
            json_data = json.load(f)
            mode = cop.split(".")[0]
            for data in json_data[mode]:
                text = data["data"]
                label = data["label"][0]
                subLabel = ""
                df = df.append(pd.DataFrame([text, label, subLabel], index = cols).T)
    df.reset_index(inplace=True, drop=True)
    return df

In [22]:
# label_list = "YN WH please plain".split()
label_list = "YN NOT".split()
label_dict = dict( zip(label_list, range(len(label_list))) )
print(label_dict)
def extract_X_y(df):
    X = []
    y = []
    for te, la in zip(df.text, df.label):
        X.append(te)
        if la not in label_dict:
            y.append(-1)
        else:
            y.append(1)
    return X, y

{'YN': 0, 'NOT': 1}


In [5]:
corpus_root = "../../corpus"
# name = "question/short"
name = "question"
data_path = "/".join([corpus_root, name]) + "/"
data_path

'../../corpus/question/'

In [7]:
df = json2data(data_path)

In [23]:
X, y = extract_X_y(df)

In [24]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

In [25]:
F = Feature()
F.make_features(X_train_str)

X_train = []
X_test = []
for i, x_t_str in enumerate( X_train_str ):
    x = F.featurization(x_t_str)
    X_train.append(x)
for i, x_t_str in enumerate( X_test_str ):
    x = F.featurization(x_t_str)
    X_test.append(x)
X_train = np.array(X_train)
X_test = np.array(X_test)

300


In [26]:
lr = LogisticRegression(solver='sag', max_iter=1000)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=1000, solver='sag')

In [27]:
y_pred = lr.predict(X_test)

In [28]:
print(label_dict)
for y_p, x_s in zip(y_pred[:30], X_test_str[:30]):
    print("{0} : {1}".format(y_p, x_s))

{'YN': 0, 'NOT': 1}
-1 : いっしょに行きたいんじゃない？
-1 : あなたがネガティブ・フォース卿ですね。
1 : 君は、僕が正直だと思うかい？
-1 : 締め切りに間に合わせることができました。
-1 : ぼくたち、学校の図書館のことで校長先生にお話ししたいのですが。
-1 : 好きだわ。
-1 : あれはりんごの木？
-1 : イケブクロ博士のところに持って帰ります。
-1 : 前の携帯からうまくデータを移行できたの？
1 : 机の中にあるのかな？
1 : それ、去年言ってなかった？
-1 : うーんと…それは秘密。
-1 : えーと……。僕は車をどこに駐車したんだっけ？
1 : 僕が9時ごろ送ったメールは読んだかい？
-1 : わたしたちはスノードロップという町へ行きたいのです。
-1 : わたしに新聞紙をいただけませんか、奏多先生？
-1 : 君の家族もいるよ、リリー。
-1 : アスカが大丈夫だといいのですが。
-1 : 今となっては遅すぎるから。
-1 : ほかに何かいかがですか？
-1 : すぐに離れたほうがいいわ。
-1 : スポーツを指導することがとても上手でね。
-1 : ふたりは、学校の図書館はだれにとっても便利というわけではないと言いました。
-1 : 誘ってくれてありがとう、勇也、でも、ごめんね。
-1 : もうひと切れいかが？
-1 : それは税込みで200円ですよ。
-1 : それはだれのバッグかな？
-1 : どうやるか見せるね。
-1 : 早く治ることを願う。
-1 : お父さんきのうの夜どこにいたの？


In [29]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[384   6]
 [ 57  37]]
accuracy =  0.8698347107438017
precision =  0.8604651162790697
recall =  0.39361702127659576
f1 score =  0.54014598540146


[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
